In [1]:
import pandas as pd
import statsmodels.formula.api as sm
insurance = pd.read_csv('https://raw.githubusercontent.com/TomdeFluiter/StudentsPerformance/refs/heads/main/insurance.csv')
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [2]:
insurance_dummies = pd.get_dummies(insurance[['sex','smoker']], dtype = int)
insurance2 = pd.concat([insurance,insurance_dummies], axis=1)
insurance2.head()

,age,sex,bmi,children,smoker,region,charges,sex_female,sex_male,smoker_no,smoker_yes
0,19,female,27.900,0,yes,southwest,16884.92400,1,0,0,1
1,18,male,33.770,1,no,southeast,1725.55230,0,1,1,0
2,28,male,33.000,3,no,southeast,4449.46200,0,1,1,0
3,33,male,22.705,0,no,northwest,21984.47061,0,1,1,0
4,32,male,28.880,0,no,northwest,3866.85520,0,1,1,0


In [3]:
model1 = sm.ols('charges~sex_male', data=insurance2).fit()
print(model1.summary())

model2 = sm.ols('charges~sex_male+smoker_yes', data=insurance2).fit()
print(model2.summary())

model3 = sm.ols('charges~sex_female+smoker_no', data=insurance2).fit()
print(model3.summary())

model4 = sm.ols('charges~smoker_yes', data=insurance2).fit()
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     4.400
Date:                Tue, 22 Oct 2024   Prob (F-statistic):             0.0361
Time:                        10:53:55   Log-Likelihood:                -14475.
No. Observations:                1338   AIC:                         2.895e+04
Df Residuals:                    1336   BIC:                         2.897e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.257e+04    470.072     26.740      0.0

In [4]:
from sklearn.preprocessing import StandardScaler
insurance3 = insurance2.copy()
insurance3[['age','bmi']] = StandardScaler().fit_transform(insurance3[['age','bmi']])

model5 = sm.ols('charges~age+bmi', data=insurance3).fit()
print(model5.summary())

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     88.60
Date:                Tue, 22 Oct 2024   Prob (F-statistic):           7.39e-37
Time:                        10:53:56   Log-Likelihood:                -14394.
No. Observations:                1338   AIC:                         2.879e+04
Df Residuals:                    1335   BIC:                         2.881e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.327e+04    311.298     42.629      0.0

In [5]:
"""
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML

Table = Stargazer([model5])

HTML(Table.render_html())

#werkt alleen in Jupyter (niet in Python)
"""

'\nfrom stargazer.stargazer import Stargazer\nfrom IPython.core.display import HTML\n\nTable = Stargazer([model5])\n\nHTML(Table.render_html())\n\n#werkt alleen in Jupyter (niet in Python)\n'

In [6]:
import numpy as np

insurance3['east_west'] = np.where(insurance3['region'].isin(['northeast', 'southeast']), 'east', 'west')

print(insurance3.head())

        age     sex       bmi  children smoker     region      charges  \
0 -1.438764  female -0.453320         0    yes  southwest  16884.92400   
1 -1.509965    male  0.509621         1     no  southeast   1725.55230   
2 -0.797954    male  0.383307         3     no  southeast   4449.46200   
3 -0.441948    male -1.305531         0     no  northwest  21984.47061   
4 -0.513149    male -0.292556         0     no  northwest   3866.85520   

   sex_female  sex_male  smoker_no  smoker_yes east_west  
0           1         0          0           1      west  
1           0         1          1           0      east  
2           0         1          1           0      east  
3           0         1          1           0      west  
4           0         1          1           0      west  


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif 
from statsmodels.tools.tools import add_constant

X = insurance3.drop(columns=['sex_female', 'smoker_no']).select_dtypes(include=[np.number]).dropna()
X = add_constant(X)

VIF_data = pd.DataFrame()
VIF_data["variable"] = X.columns
VIF_data["VIF"] = [vif(X,i)
    for i in range(len(X.columns))]
print(VIF_data)

     variable       VIF
0       const  4.718352
1         age  1.372390
2         bmi  1.119867
3    children  1.011153
4     charges  3.995565
5    sex_male  1.008991
6  smoker_yes  3.526459


In [8]:
model6 = sm.ols('charges~east_west', data=insurance3).fit()
print(model6.summary())

model7 = sm.ols('charges~east_west+smoker_yes+bmi', data=insurance3).fit()
print(model7.summary()) #smoking & bmi

                            OLS Regression Results                            
Dep. Variable:                charges   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     6.829
Date:                Tue, 22 Oct 2024   Prob (F-statistic):            0.00907
Time:                        10:53:56   Log-Likelihood:                -14474.
No. Observations:                1338   AIC:                         2.895e+04
Df Residuals:                    1336   BIC:                         2.896e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          1.411e+04    460.68

In [9]:
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean, absolute
from sklearn.preprocessing import MinMaxScaler

insurance4 = insurance3.dropna()
y = insurance4['charges']
X = insurance4.select_dtypes(exclude=['object']).drop('charges', axis=1)
X = pd.get_dummies(X, drop_first = True)

cv = RepeatedKFold(n_splits=5, random_state=420)

lm = LinearRegression()
lmscores = cross_val_score(lm,X,y,scoring='neg_mean_absolute_error', cv=cv)

lmMAE = mean(absolute(lmscores))
print('the average prediction error with full data is %.0f' % lmMAE)

the average prediction error with full data is 4221


In [10]:
insurance4 = insurance3.dropna()
y2 = insurance4['charges']
X2 = insurance4.select_dtypes(exclude=['object']).drop(['charges','sex_male','sex_female'], axis=1)
X2 = pd.get_dummies(X2, drop_first = True)

cv = RepeatedKFold(n_splits=5, random_state=420)

lm = LinearRegression()
lmscores = cross_val_score(lm,X2,y2,scoring='neg_mean_absolute_error', cv=cv)

lmMAE = mean(absolute(lmscores))
print('the average prediction error with full data is %.0f' % lmMAE)

the average prediction error with full data is 4203


In [11]:
columns = X.columns
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=columns)

In [12]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
alpha_range = np.arange(start=1,stop=100,step=1)

cv = RepeatedKFold(n_splits=5)
param = {'alpha':alpha_range}
LassoModel = Lasso()

LassoM = GridSearchCV(LassoModel,param_grid=param,scoring='neg_mean_absolute_error', cv=cv)
LassoM.fit(X,y)
print("Best Alpha: ",LassoM.best_params_['alpha'])

RidgeModel = Ridge()

RidgeM = GridSearchCV(RidgeModel, param_grid=param, scoring='neg_mean_absolute_error', cv=cv)
RidgeM.fit(X, y)
print("Best Alpha: ", RidgeM.best_params_['alpha'])

/usr/local/python/3.12.1/lib/python3.12/site-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Alpha:  80
Best Alpha:  8


In [13]:
RidgeModel = Ridge(alpha=8)
scoresridge = cross_val_score(RidgeModel,X,y,scoring='neg_mean_absolute_error', cv=cv)
ridgeMAE = mean(absolute(scoresridge))
print('the average prediction error with full data is %.0f' % ridgeMAE)

the average prediction error with full data is 4157


In [14]:
LassoModel = Lasso(alpha=80)
scoreslasso = cross_val_score(LassoModel,X,y,scoring='neg_mean_absolute_error', cv=cv)
lassoMAE = mean(absolute(scoreslasso))
print('the average prediction error with full data is %.0f' % lassoMAE)

the average prediction error with full data is 4151
